<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Привет еще раз. Спасибо, что доделал работу. Оформление комментариев по работе сохраняется. Только обозначим, что это вторая итерация. 

</div>

<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>

Привет, Даниил! Спасибо, что прислал задание:) Меня зовут Слепцов Артем и я буду проверять твой проект) Ты проделал большую работу над проектом. Он выполнен уже на достойном уровне. Однако есть моменты, которые еще можно улучшить. Будет здорово, если ты, надеюсь, не против, если я буду на ты, будешь отвечать на комментарии и участвовать в диалоге. Если обращение на ты неприемлемо, то прошу сообщить. 

Мои комментарии обозначены пометкой **Комментарий ревьюера**. Далее в файле ты сможешь найти их в похожих ячейках:
    
<div class="alert alert-success">Если фон комментария зелёный - всё сделано правильно. Рекомендации укажу таким же цветом;</div>
        
<div class="alert alert-warning">Оранжевый - некритичные замечания. Если таких замечаний меньше трех - проект может быть принят без их отработки;</div>
        
<div class="alert alert-danger">Красный - нужно переделать. </div>
        
Не удаляй эти комментарии и постарайся учесть их в ходе выполнения данного проекта. Свои же комментарии ты можешь обозначать любым заметным способом. 

</div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Вступление в работу очень важно. Так ты поясняешь то, чему она посвящена. Стоит также описать те данные, с которыми мы работаем. 

</div>

## Шаг 1. Откройте файл с данными и изучите общую информацию

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>

Я заметил, что ячейки в твоей тетрадке идут не по порядку. Перед отправкой работы рекомендую перезапускать ноутбук, чтобы убедиться, что все ячейки выполняются корректно.

</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Перезапуск позволяет обнаружить и исправить скрытые в коде ошибки. 

</div>

In [1]:
# Считывание файла с данными и общая информация о типах и размерности данных 
import pandas as pd
import numpy as np
data=pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
# Артефакты в данных, которые можно удалить,т.к. они имеют случайный характер
print(data.loc[data.children<0].shape)
data=data.loc[~data.children<0]

(47, 12)


In [4]:
# Артефакты переменной dob_years, которые можно удалить, т.к. они имеют случайный характер
print(data.loc[data['dob_years']<18].shape)
data=data.loc[~data['dob_years']<18]

(101, 12)


In [5]:
data['days_employed'].loc[data['days_employed']>0].shape[0]/(data['days_employed'].shape[0])*100

16.007077009032496

In [6]:
data['days_employed'].loc[data['days_employed']<0].shape[0]/(data['days_employed'].shape[0])*100

73.884905484682

In [7]:
# Отрицательный стаж 
data.loc[data['days_employed']<0]['income_type'].value_counts()

сотрудник          9989
компаньон          4569
госслужащий        1308
предприниматель       1
студент               1
в декрете             1
Name: income_type, dtype: int64

In [8]:
# Положительный стаж только у безработных и пенсионеров
data.loc[data['days_employed']>0]['income_type'].value_counts()

пенсионер      3436
безработный       2
Name: income_type, dtype: int64

**Вывод**

Датасет размерностью (21525,12). **days_employed** имеет отрицательные и положительные значения и и формат ***float64***.   NaN значения есть в 2 полях: ***days_employed*** и ***total_income***. Для ***days_employed*** : положительных значений ~ **16%**,отрицательных - **74%**  и **10%** пропусков. Отрицательные значения трудового стажа связаны с социальным статусом займщика - у пенсионеров и безработных он положительный (с чрезвычайно высокими - более 1000 лет стажа, значениями).  Отсутствующие значения следует заменить на медианные для каждой социальной группы. 
Также обнаружены *47 (~0.22%)* отрицательных значений переменной **children** и *101(~0.46%)* нулевых значений переменной **dob_years** что,  является артефактом.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Подумай также о возможных причинах появления пропусков, а также о том, являются ли они случайными или нет. 

Первый взгляд на таблицу выполнен.

</div>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
data.groupby(by='income_type').median()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_type,,,,,,,
безработный,0.5,366413.652744,38.0,0.5,0.5,0.5,131339.751676
в декрете,2.0,-3296.759962,39.0,1.0,0.0,1.0,53829.130729
госслужащий,0.0,-2689.368353,40.0,1.0,0.0,0.0,150447.935283
компаньон,0.0,-1548.721148,39.0,1.0,0.0,0.0,172466.509253
пенсионер,0.0,365249.346345,60.0,1.0,0.0,0.0,118557.179415
предприниматель,0.0,-520.848083,42.5,0.0,0.5,0.0,499163.144947
сотрудник,0.0,-1572.762276,39.0,1.0,0.0,0.0,142587.588976
студент,0.0,-578.751554,22.0,0.0,4.0,0.0,98201.625314


In [10]:
# Заполняем NaN-значения на медианные для каждой  группы по income_type
data['total_income'] = data.groupby(['income_type'])['total_income'].apply(lambda x: x.fillna(x.median()))

In [11]:
data['days_employed'] = data.groupby(['income_type'])['days_employed'].apply(lambda x: x.fillna(x.median()))

**Вывод**

Пропущенные значения были заменены на медианные ( чтобы не искажать распределение дохода ) по каждой каждой группе **income_type**, т.к. доход и стаж коррелируют с социальным статусом человека. 

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Соглашусь, тип занятости сильно влияет на получаемый доход. Здорово, что ты заметил данную особенность и смог обработать пропуски, исходя из данной зависимости. 

</div>

### Замена типа данных

In [12]:
data['days_employed']=data['days_employed'].astype('int')

In [13]:
data['total_income'] = data['total_income'].astype('int')

In [14]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [15]:
data['children'].value_counts()

0     14149
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

**Вывод**

Заменен тип данных у предикторов **days_employed** и **total_income** c *float64* на *int64* с помощью метода astype(), чтобы при дальнейшем анализе не вводить в заблуждение, т.к. стаж и доход измеряют в целых днях и рублях. Т 

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Исходя из общей информации о таблице мы видим, что два столбца имеют вещественный тип данных, который и надо заменить на целочисленный. Эта замена произведена только для стажа. Доход также стоит сделать целочисленным показателем. 5 знаков после запятой нам явно не нужны. Также хорошо бы освоить метод to_numeric() для будущих работ. 

</div>

### Обработка дубликатов

In [16]:
# Для начала  приведём все категориальные данные к одному регистру
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null int64
purpose             21478 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [18]:
# Функция для приведения значений категориальных полей к нижнему регистру
def lower_reg(list):
    for el in list:
        data[el]=data[el].str.lower()

In [19]:
# Список с категориальными переменными
category_fields=['education','family_status','gender','income_type','purpose']
lower_reg(category_fields)

In [20]:
data.duplicated().sum()

71

In [21]:
data.drop_duplicates(inplace=True)

**Вывод**

Все категориальные предикторы **education, family_status, gender, income_type, purpose** приведены к нижнему регистру. Затем, с помощью методов *duplicated()* и *sum()* выявлены 71 дубликата (повторяющиеся строки в наборе данных).Эти дубли вызваны разными стилями ведения базы данных, например: использование заглавной буквы или ведение записи в нижнем регистре, включение CAPS LOCK и т.д. Затем, при помощи метода *drop_duplicates()*, дубликатные записи были удалены из dataframe

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Дубликаты вызывают смещение финальных результатов, поэтому их и надо удалять. Конечно, в данных нет уникального идентификатора, однако надо уметь удалять дубликаты. Метод drop_duplicates() использован верно.

</div>

### Лемматизация и Категоризация данных

In [22]:
# На какие же вообще цели берут люди кредит - огласите весь список, пожалуйста!
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           764
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
операции с жильем                         651
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     649
жилье                                     645
покупка жилья                             643
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              632
операции со своей недвижимостью           624
строительство жилой недвижимости          621
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              606
покупка жилой недвижимости                604
на покупку своего автомобиля              504
заняться высшим образованием      

#### Исходя из этого списка, можно предположить, что люди чаще всего берут кредиты на свадьбу. Но это будет грубой ошибкой, потому что в переменной *purpose* одна и та же цель кредита может быть записана по-разному, например: "операции с недвижимостью"  и  " операции с жильём".

In [23]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

#### Можно выделить 4 общих категории выдачи займа: "свадьба","недвижимость","автомобиль","образование". Для того чтобы сгруппировать заявки по этим категориям - проведём лемматизацию, т.е. выявим словарную форму в каждом значении переменной *purpose* и каждой основной лемме (характеризующей цель кредита) будем сопоставлять новые категории. Записывать значения будем в поле  *pur_group*

In [24]:
from pymystem3 import Mystem
m = Mystem()

In [25]:
def purp_group(series):
    for ind,row in series.iteritems():
        lemm=m.lemmatize(row)
        if 'свадьба' in lemm:
            data.loc[ind,'pur_group']='свадьба'
        elif 'автомобиль' in lemm:
            data.loc[ind,'pur_group']='автомобиль'
        elif  'недвижимость' in lemm:
            data.loc[ind,'pur_group']='недвижимость'
        elif 'жилье' in lemm:
            data.loc[ind,'pur_group']='недвижимость'
        elif 'образование' in lemm:
            #print(lemm)
            data.loc[ind,'pur_group']='образование'
        else:
            data.loc[ind,'pur_group']='неизвестно'

In [26]:
purp_group(data['purpose'])

In [27]:
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,pur_group
21515,1,-467,28,среднее,1,женат / замужем,0,f,сотрудник,1,109486,заняться образованием,образование
21516,0,-914,42,высшее,0,женат / замужем,0,f,компаньон,0,322807,покупка своего жилья,недвижимость
21517,0,-404,42,высшее,0,гражданский брак,1,f,компаньон,0,178059,на покупку своего автомобиля,автомобиль
21518,0,373995,59,среднее,1,женат / замужем,0,f,пенсионер,0,153864,сделка с автомобилем,автомобиль
21519,1,-2351,37,ученая степень,4,в разводе,3,m,сотрудник,0,115949,покупка коммерческой недвижимости,недвижимость
21520,1,-4529,43,среднее,1,гражданский брак,1,f,компаньон,0,224791,операции с жильем,недвижимость
21521,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,сделка с автомобилем,автомобиль
21522,1,-2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,недвижимость
21523,3,-3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля,автомобиль
21524,2,-1984,40,среднее,1,женат / замужем,0,f,сотрудник,0,82047,на покупку автомобиля,автомобиль


In [28]:
data['pur_group'].value_counts()

недвижимость    10787
автомобиль       4295
образование      4003
свадьба          2322
Name: pur_group, dtype: int64

**Вывод**

Выявлены 4 общих категории для цели кредита: "свадьба","недвижимость","автомобиль","образование". Проведена лемматизация значений переменной **purpose**. Новые категории записаны в переменную **pur_group** 

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Круто, ты сделал самую сложную часть проекта. Использовал нужные методы, и даже провел группировку с помощью собственной функции. Выбор основных целей кредитов обоснован. 

</div>

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

В данных есть словари. Попробуй их обнаружить и выделить. Это довольно распространенный способ хранения информации. Мне важно понять, что он тобой освоен. Заметь, что в основной таблице мы имеем по 2 параметра с образованием и семейным статусом. Именно по ним и стоит выделить словари. 

</div>

In [29]:
# Словарь с образованием
dict(data[['education','education_id']].drop_duplicates().to_dict('split')['data'])

{'высшее': 0,
 'среднее': 1,
 'неоконченное высшее': 2,
 'начальное': 3,
 'ученая степень': 4}

In [30]:
# Словарь с семейным статусом
dict(data[['family_status','family_status_id']].drop_duplicates().to_dict('split')['data'])

{'женат / замужем': 0,
 'гражданский брак': 1,
 'вдовец / вдова': 2,
 'в разводе': 3,
 'не женат / не замужем': 4}

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Отлично, словари выделены. Теперь мы можем убрать текстовые параметры из основной таблицы, что увеличит эффективность работы с данными. 

</div>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
# Группируем данные по количеству детей у займщиков и суммируем по полю debt. Далее нормируем заёмщиков на их количество в каждой группе, т.к. в абсолютном выражении многодетных заёмщиков меньше.
pd.DataFrame(round(data.groupby(by='children')['debt'].sum()/data.groupby(by='children')['debt'].count()*100,2))

,debt
children,
0,7.54
1,9.23
2,9.45
3,8.18
4,9.76
5,0.00
20,10.53


**Вывод**

Согласно данным, существует слабая зависимость между количеством детей и выплатой кредита в срок: люди без детей чуть чаще возвращают кредит, чем заёмщики с детьми, т.к. с финансовой точки зрения, дети являются дополнительным обременяющим фактором, особенно в юном возрасте. Но в среднем, детей заводят финансово самостоятельные люди, т.е. они планируют рождение детей и возросшие материальные издержки. 

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Вывод не противоречит полученному результату. 

</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
# Группируем данные семейному положению и суммируем по полю debt.
# Далее нормируем заёмщиков на их количество в каждой группе, чтобы нивелировать разницу в количестве
pd.DataFrame(round(data.groupby(by='family_status')['debt'].sum()/data.groupby(by='family_status')['debt'].count()*100,2))

,debt
family_status,
в разводе,7.14
вдовец / вдова,6.60
гражданский брак,9.36
женат / замужем,7.55
не женат / не замужем,9.77


**Вывод**

Зависимость есть: *не женатые / не замужние* заёмщики и заёмщики состоящие в *гражданском браке* реже возвращают кредит, чем остальные. Так как в брак, в среднем, вступают в более зрелом возрасте, не говоря уже об овдовевших заёмщиках, то такие заёмщики более ответственные и дисциплинированные.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Подумай, почему мы получаем такой результат. Можем ли мы тут иметь скрытую зависимость от возраста клиентов?

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
# Поделим всех заёмщиков на 10 равных групп (децилей), в зависимости от уровня дохода
data['inc_group']=pd.qcut(data['total_income'],10,duplicates='raise',labels=[0,1,2,3,4,5,6,7,8,9])

In [34]:
# [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]
# где 0 - самые бедный дециль заёмщиков, 9 - самый обеспеченный дециль заёмщиков
round(pd.pivot_table(data,values='debt',index='inc_group',aggfunc=np.mean)*100,2)

,debt
inc_group,
0,7.33
1,8.73
2,8.36
3,8.50
4,9.08
5,8.35
6,8.45
7,8.41
8,6.96


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Здорово, чтоо ты применил данный метод. Так мы получаем равные по размеру группы и можем построить надежные выводы. 

</div>

**Вывод**

Однозначной взаимосвязи между возвратом кредита в срок и уровнем дохода нет. Это объясняется тем, что у людей с ростом доходов растут и потребности, т.е. они берут кредит на те вещи, за которые не могут себе позволить заплатить сразу всю сумму - автомобили премиальных сегментов, элитную недвижимость в центральных районах городов.

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Исходя из чего выбран именно такой способ категоризации столбца с доходами? Не во всех группах содержится достаточное число клиентов для построения по ним надежных выводов. Исправь способ выделения групп таким образом, чтобы в каждой содержалось достаточное число клиентов. Делать выводы по 25 клиентам или по 200 клиентам может быть крайне опасно. 
</div>
<div class="alert alert-success">
    
Лучше использовать разбиение по квантилям данных (метод qcut). Тогда группы получатся одинакового размера. Или же можно узнать в интернете процентный состав общества по доходам и затем уже категоризовать столбец с доходами по найденному соотношению с помощью персентилей. 

</div>

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
#Группируем данные по цели займа (pur_group) и усредняем по полю debt
round(pd.pivot_table(data,values='debt',index='pur_group',aggfunc=np.mean)*100,2)

,debt
pur_group,
автомобиль,9.36
недвижимость,7.25
образование,9.24
свадьба,8.01


**Вывод**

Кредиты взятые на недвижимость - самые беспроигрышные для банка, оно и понятно, ведь в таком случае имущество переходит от заёмщика в собственность банка. А кредиты, взятые на автомобиль и образование - самые проблемные, как для банка, так и для заёмщика: по этим кредитам чаще всего возникают невозвраты. 

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

На данном шаге рекомендуется использовать метод сводных таблиц (pivot_table()). Однако он является аналогом метода groupby(). Изучи метод сводных таблиц и примени его хотя бы в одном пункте анализа.

</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Метод сводных таблиц освоен. Он нам сильно пригодится в дальнейших проектах.

</div>

## Шаг 4. Общий вывод

Исследован набор данных по заёмщикам банка. Проведена обработка и заполнение 2174 NaN-значений в переменных ***days_employed*** и ***total_income*** на медианные, также у этих переменных были заменены типы данных с дробных значений *float* на целочисленные *int*. Удалена 71 дублирующая запись. Произведена лемматизация и категоризация целей займа ***purpose*** - выделены 4 основных категории: 
- займы на покупку автомобиля;
- займы на сделки с недвижимостью;
- займы на организацию свадеб, и займы на получение образования.

Произведены исследования:
- **на наличие взаимосвязи между наличием детей и возвратом кредита в срок**

Согласно данным, существует слабая зависимость между количеством детей и выплатой кредита в срок: люди без детей чуть чаще возвращают кредит, чем заёмщики с детьми, т.к. с финансовой точки зрения, дети являются дополнительным обременяющим фактором, особенно в юном возрасте. Но в среднем, детей заводят финансово самостоятельные люди, т.е. они планируют рождение детей и возросшие материальные издержки. 
- **на связь между семейным положением и возвратом кредита в срок**

*не женатые / не замужние* (9.77 % невозвратов) заёмщики и заёмщики состоящие в *гражданском браке* (9.36 % дефолтов) реже возвращают кредит, чем остальные (менее 7.55% невозвратов). Так как в брак, в среднем, вступают в более зрелом возрасте, не говоря уже об овдовевших заёмщиках, то такие заёмщики более ответственные и дисциплинированные. Таким образом наблюдается скрытая зависимость от возраста заёмщика.
- **зависимость между уровнем дохода и возвратом кредита в срок**

Были взяты 10 децилей заёмщиков по уровню дохода. Внутри этих групп посчитан процент дефолтов.
Однозначной взаимосвязи между возвратом кредита в срок и уровнем дохода нет. У самых бедных заёмщиков 7.33 % невыплат, а у самых богатых - 7.05%, при этом у заёмщиков со средним уровнем дохода вероятность наступления дефолта достигает 9.08 %.Это объясняется тем, что у людей с ростом доходов растут и потребности, т.е. они берут кредит на те вещи, за которые не могут себе позволить заплатить сразу всю сумму - автомобили премиальных сегментов, элитную недвижимость в центральных районах городов и т.д. 
- **как разные цели кредита влияют на его возврат в срок.**

Кредиты взятые на недвижимость - самые беспроигрышные для банка (7.25 % дефолтов), так как, в этом случае имущество переходит от заёмщика в собственность банка. А кредиты, взятые на автомобиль и образование - самые проблемные, как для банка, так и для заёмщика: по этим кредитам чаще всего возникают невозвраты - в 9.36 % и 9.24 % случаев, соответственно.

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. В нем можно приводить полученные в ходе работы значения. Также можно расписать все, что было сделано в работе.

</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Выводы описаны грамотно. Приведены ответы на главные вопросы проекта. В выводах можно приводить полученные ранее значения, правильно. Здорово, что по каждому пункту работы приведен вывод.     

</div>

<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>

- Выдели словари в данных;
- Категоризацию по доходу стоит делать иначе. Смотри данный пункт подробнее;
- Изучи и используй в анализе метод сводных таблиц;
- Финальный вывод стоит писать подробно по результатам проделанной работы.

Ты проделал большую работу, молодец! Исправь отмеченные выше помарки. Думаю, сделать это тебе не составит труда. Жду твою работу :)

</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера 2</b></font>

Помарки исправлены, и теперь работа выполнена хорошо. Ты здорово поработал, молодец. Продолжай в том же духе! Успехов в дальнейших проектах :)

</div>
   

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.